In [1]:
!pip install --upgrade pip transformers adapters scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 100.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. 

In [2]:
import numpy as npimport pandas as pdimport torchimport osimport jsonfrom sklearn.model_selection import train_test_splitfrom sklearn.utils.class_weight import compute_class_weightfrom sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_reportfrom peft import get_peft_model, LoraConfig, TaskTypefrom torch.utils.data import Datasetfrom transformers import ( XLMRobertaForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer)

2025-12-04 22:34:07.371012: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764887647.531632      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764887647.583116      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
INPUT_DF_PATH = "/kaggle/input/depressive-data/depressive_data_with_entities.csv"INPUT_STOPWORDS_PATH = "/kaggle/input/stopwords-ru/stopwords-ru.txt"MODEL_NAME = "DeepPavlov/rubert-base-cased"RANDOM_STATE = 42

In [4]:
df = pd.read_csv(INPUT_DF_PATH, encoding='utf-8')df.head()

/tmp/ipykernel_20/470166782.py:1: DtypeWarning: Columns (1,2,3,4,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(INPUT_DF_PATH, encoding='utf-8')
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in great

,clean_text,label,age,text_length,word_count,language,entities,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45
0,–ö–æ–≥–¥–∞-—Ç–æ —è –±—ã–ª –¥–æ–±—Ä—ã–º —Ä–æ–º–...,1,32.0,1699,272,ru,"[('–∞–ø–∞—Ç–∏—è', 'SYMPTOM')]",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,–ó–¥—Ä–∞—Å—Ç–≤—É–π—Ç–µ! –Ø –∫–∞–∂–¥—ã–π –¥–µ–Ω...,1,28.0,769,121,ru,[],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,–£ –º–µ–Ω—è –ø—Ä–æ–±–ª–µ–º—ã —Å –¥–µ–≤—É—à–∫–æ...,1,16.0,501,90,ru,"[('–ø–æ–∫–æ–Ω—á–∏—Ç—å —Å —Å–æ–±–æ–π', 'SUICIDA...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,–í—Å—è –º–æ—è –∂–∏–∑–Ω—å —ç—Ç–æ –æ–¥–∏–Ω —Å–ø–...,1,32.0,236,45,ru,"[('—É—Å—Ç–∞–ª', 'EMOTIONAL_STATE')]",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,–Ø —Ö–æ—á—É —É—Å–Ω—É—Ç—å –∏ –Ω–µ –ø—Ä–æ—Å–Ω—É—...,1,14.0,2074,365,ru,"[('–¥–µ–ø—Ä–µ—Å—Å–∏—è', 'CONDITION'), ('–¥–µ–ø...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df = df[['clean_text', 'label']]df.head()

,clean_text,label
0,–ö–æ–≥–¥–∞-—Ç–æ —è –±—ã–ª –¥–æ–±—Ä—ã–º —Ä–æ–º–...,1
1,–ó–¥—Ä–∞—Å—Ç–≤—É–π—Ç–µ! –Ø –∫–∞–∂–¥—ã–π –¥–µ–Ω...,1
2,–£ –º–µ–Ω—è –ø—Ä–æ–±–ª–µ–º—ã —Å –¥–µ–≤—É—à–∫–æ...,1
3,–í—Å—è –º–æ—è –∂–∏–∑–Ω—å —ç—Ç–æ –æ–¥–∏–Ω —Å–ø–...,1
4,–Ø —Ö–æ—á—É —É—Å–Ω—É—Ç—å –∏ –Ω–µ –ø—Ä–æ—Å–Ω—É—...,1


In [6]:
try: with open(INPUT_STOPWORDS_PATH, "r", encoding="utf-8") as f: stopwords_ru = [line.strip() for line in f if line.strip()]except FileNotFoundError: print(f"Error: The file '{file_path}' was not found. Please ensure the file exists in your Google Drive.")except Exception as e: print(f"An error occurred while loading the file: {e}")

In [7]:
def remove_stopwords(text, stop_words): """Removes stop words from a string.""" if isinstance(text, str): words = text.split() filtered_words = [word for word in words if word.lower() not in stop_words] return ' '.join(filtered_words) return ""# Apply stop word removal to the 'text' column of filtered_dfprint("Removing stop words from filtered_df...")df['text_no_stopwords'] = df['clean_text'].apply(lambda x: remove_stopwords(x, set(stopwords_ru)))print("DataFrame after removing stop words:")display(df.head())

Removing stop words from filtered_df...
DataFrame after removing stop words:


,clean_text,label,text_no_stopwords
0,–ö–æ–≥–¥–∞-—Ç–æ —è –±—ã–ª –¥–æ–±—Ä—ã–º —Ä–æ–º–...,1,–ö–æ–≥–¥–∞-—Ç–æ —è –±—ã–ª –¥–æ–±—Ä—ã–º —Ä–æ–º–...
1,–ó–¥—Ä–∞—Å—Ç–≤—É–π—Ç–µ! –Ø –∫–∞–∂–¥—ã–π –¥–µ–Ω...,1,–ó–¥—Ä–∞—Å—Ç–≤—É–π—Ç–µ! –Ø –∫–∞–∂–¥—ã–π –¥–µ–Ω...
2,–£ –º–µ–Ω—è –ø—Ä–æ–±–ª–µ–º—ã —Å –¥–µ–≤—É—à–∫–æ...,1,–£ –º–µ–Ω—è –ø—Ä–æ–±–ª–µ–º—ã —Å –¥–µ–≤—É—à–∫–æ...
3,–í—Å—è –º–æ—è –∂–∏–∑–Ω—å —ç—Ç–æ –æ–¥–∏–Ω —Å–ø–...,1,–í—Å—è –º–æ—è –∂–∏–∑–Ω—å —ç—Ç–æ –æ–¥–∏–Ω —Å–ø–...
4,–Ø —Ö–æ—á—É —É—Å–Ω—É—Ç—å –∏ –Ω–µ –ø—Ä–æ—Å–Ω—É—...,1,–Ø —Ö–æ—á—É —É—Å–Ω—É—Ç—å –∏ –Ω–µ –ø—Ä–æ—Å–Ω—É—...


## Russian Data Split

## Russian Data Preprocess

In [8]:
# Drop any rows with missing textdf.dropna(inplace=True)# If labels should be integers, convert or drop invalid rowsdf['label'] = pd.to_numeric(df['label'], errors='coerce')df.dropna(subset=['label'], inplace=True)df['label'] = df['label'].astype(int)# Split into train/validation and testtrain_val_df, test_df = train_test_split( df, test_size=0.15, random_state=RANDOM_STATE, stratify=df['label'])# Split train/validation into train and validationtrain_df, val_df = train_test_split( train_val_df, test_size=0.15, # 15% of the 85% random_state=RANDOM_STATE, stratify=train_val_df['label'])print(f"Training set size: {len(train_df)}")print(f"Validation set size: {len(val_df)}")print(f"Test set size: {len(test_df)}")

Training set size: 43061
Validation set size: 7599
Test set size: 8941


## Russian Tokenizer

In [9]:
# Load the XLM-RoBERTa tokenizertokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base", use_fast=True)class TextClassificationDataset(Dataset): def __init__(self, dataframe, tokenizer, max_length=256): self.tokenizer = tokenizer self.texts = dataframe['clean_text'].tolist() self.labels = dataframe['label'].tolist() self.max_length = max_length def __len__(self): return len(self.texts) def __getitem__(self, idx): text = self.texts[idx] label = self.labels[idx] encoding = self.tokenizer( text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt' ) return { 'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'labels': torch.tensor(label, dtype=torch.long) }

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

## Load Model and Data

In [10]:
# 1. Load your Russian test dataset# (This assumes 'test_df' and 'tokenizer' are already loaded)print(f"Creating Russian val & test dataset with {len(val_df)} and {len(test_df)} examples...")train_dataset = TextClassificationDataset(train_df, tokenizer)val_dataset = TextClassificationDataset(val_df, tokenizer)test_dataset = TextClassificationDataset(test_df, tokenizer)# 2. Load the base XLM-R model architectureprint("Loading base XLM-RoBERTa model...")model = XLMRobertaForSequenceClassification.from_pretrained( "xlm-roberta-base", num_labels=df['label'].nunique())# 3. Re-create the *original* adapter config# This MUST match the config you used for your English model (r=8)peft_config = LoraConfig( task_type=TaskType.SEQ_CLS, r=8, lora_alpha=16, lora_dropout=0.1, target_modules=['query', 'value', 'key', 'dense'])model = get_peft_model(model, peft_config)print(" Model architecture created.")

Creating Russian val & test dataset with 7599 and 8941 examples...
Loading base XLM-RoBERTa model...


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model architecture created.


## Load BEST English-Trained Adapter

In [11]:
# Setup devicedevice = torch.device('cuda' if torch.cuda.is_available() else 'cpu')print(f" Using device: {device}")

✅ Using device: cuda


In [12]:
# YOU MUST CHANGE THIS PATH# This is the 'best_model_checkpoint' path from your *first* (English)# training run, which you saved as a Kaggle dataset.ENGLISH_CHECKPOINT_PATH = "/kaggle/input/checkpoints/checkpoint-100000"print(f"Loading English adapter weights from: {ENGLISH_CHECKPOINT_PATH}")model.load_adapter(ENGLISH_CHECKPOINT_PATH, adapter_name="default")model.to(device)print(" English weights loaded successfully.")

Loading English adapter weights from: /kaggle/input/checkpoints/checkpoint-100000
✅ English weights loaded successfully.


## Zero-Shot Prediction

In [13]:
# We only need a temporary Trainer to run .predict()temp_args = TrainingArguments( output_dir="/kaggle/working/zero_shot_eval", per_device_eval_batch_size=32, # Use a batch size for evaluation report_to="none")trainer = Trainer( model=model, args=temp_args)print(" Running zero-shot prediction on Russian test set...")prediction_output = trainer.predict(test_dataset)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


🚀 Running zero-shot prediction on Russian test set...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


In [14]:
# (Make sure your get_classification_report function is defined in a cell above)# Here it is again for safety:def get_classification_report(prediction_output, target_names): logits = prediction_output.predictions predictions = np.argmax(logits, axis=1) true_labels = prediction_output.label_ids report_str = classification_report( true_labels, predictions, target_names=target_names ) return report_str, predictions, true_labels# Get and print the final reportprint("\n--- Zero-Shot Performance Report ---")report_str, _, _ = get_classification_report( prediction_output, target_names=['control', 'depression'] # Or your correct labels)print(report_str)


--- Zero-Shot Performance Report ---
              precision    recall  f1-score   support

     control       0.51      1.00      0.68      4579
  depression       1.00      0.00      0.01      4362

    accuracy                           0.51      8941
   macro avg       0.76      0.50      0.34      8941
weighted avg       0.75      0.51      0.35      8941



## Few-Shot Prediction

In [15]:
# The sample sizes from your planN_SAMPLES = [50, 100, 250, 500, 1000]# This will store our final resultsfew_shot_results = {}# Create the full test dataset one timetest_dataset = TextClassificationDataset(test_df, tokenizer)

In [16]:
def compute_metrics(eval_pred): predictions, labels = eval_pred predictions = np.argmax(predictions, axis=1) # Get metrics per class (e.g., 0=control, 1=depression) precision, recall, f1, _ = precision_recall_fscore_support( labels, predictions, average=None, labels=[0, 1] ) # Get overall weighted metrics acc = accuracy_score(labels, predictions) weighted_f1 = precision_recall_fscore_support( labels, predictions, average='weighted' )[2] # [2] is the f1-score return { 'accuracy': acc, 'f1_weighted': weighted_f1, 'f1_control': f1[0], 'f1_depression': f1[1] # The metric we want to optimize }class WeightedLossTrainer(Trainer): def __init__(self, *args, class_weights_tensor=None, **kwargs): """ Initializes the WeightedLossTrainer. Args: class_weights_tensor (torch.Tensor): The tensor of class weights. """ # Call the parent Trainer's __init__ method super().__init__(*args, **kwargs) # Store the class weights tensor self.class_weights_tensor = class_weights_tensor def compute_loss(self, model, inputs, return_outputs=False, **kwargs): """ Overrides the compute_loss method to use class weights. """ labels = inputs.pop("labels") outputs = model(**inputs) logits = outputs.get("logits") # Use the stored class weights tensor # Move it to the same device as the logits weights = self.class_weights_tensor.to(logits.device) # Manually compute the loss using our class weights loss_fct = torch.nn.CrossEntropyLoss(weight=weights) loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1)) return (loss, outputs) if return_outputs else loss

In [17]:
# 2. START THE FEW-SHOT LOOPfor n in N_SAMPLES: print(f"\n--- STARTING FEW-SHOT RUN: N={n} ---") # A. Create the few-shot dataset # We use train_test_split to get a small, stratified sample _, few_shot_df = train_test_split( train_df, test_size=n, # Get N samples random_state=RANDOM_STATE, stratify=train_df['label'] ) few_shot_dataset = TextClassificationDataset(few_shot_df, tokenizer) print(f"Created dataset with {n} samples.") print(few_shot_df['label'].value_counts()) # Show the balance # B. Calculate class weights for this tiny sample # This is critical as the sample is very imbalanced sample_labels = few_shot_df['label'].tolist() class_weights = compute_class_weight( class_weight='balanced', classes=np.unique(sample_labels), y=sample_labels ) class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device) print(f"Calculated class weights for N={n}: {class_weights_tensor}") # C. Load the model and English adapter # We re-load the model *inside the loop* to start from the # same English checkpoint every time. # 1. Load base model model = XLMRobertaForSequenceClassification.from_pretrained( "xlm-roberta-base", num_labels=train_df['label'].nunique() ) # 2. Define LoRA config (must match English model) peft_config = LoraConfig( task_type=TaskType.SEQ_CLS, r=8, lora_alpha=16, lora_dropout=0.1, target_modules=['query', 'value', 'key', 'dense'] ) model = get_peft_model(model, peft_config) # 3. Load the English adapter weights model.load_adapter(ENGLISH_CHECKPOINT_PATH, adapter_name="default") model.to(device) print(f"Loaded English checkpoint: {ENGLISH_CHECKPOINT_PATH}") # D. Set Training Arguments for few-shot training_args = TrainingArguments( output_dir=f"/kaggle/working/few_shot_N-{n}", num_train_epochs=5, # Train for a few epochs per_device_train_batch_size=4, gradient_accumulation_steps=2, logging_steps=10, eval_strategy="epoch", # Evaluate every epoch save_strategy="epoch", load_best_model_at_end=True, metric_for_best_model="eval_f1_depression", greater_is_better=True, fp16=True, report_to="none" ) # E. Initialize and train trainer = WeightedLossTrainer( model=model, args=training_args, train_dataset=few_shot_dataset, eval_dataset=val_dataset, # Evaluate on the full validation set compute_metrics=compute_metrics, # We pass our dynamically calculated weights class_weights_tensor=class_weights_tensor ) print(f"Starting training for N={n}...") trainer.train() # F. Evaluate on the TEST set print(f"Training for N={n} complete. Evaluating on test set...") prediction_output = trainer.predict(test_dataset) # Store the metrics few_shot_results[n] = prediction_output.metrics # G. PRINT CLASSIFICATION REPORT FOR THIS N print(f"\n--- Classification Report for N={n} ---") report_str, predictions, true_labels = get_classification_report( prediction_output, target_names=['control', 'depression'] # Adjust if needed ) print(report_str) print(f"\n--- Summary Results for N={n} ---") print(prediction_output.metrics) print(f"--- FINISHED RUN: N={n} ---")# 3. PRINT FINAL SUMMARYprint("\n\n--- 🏆 FINAL FEW-SHOT RESULTS SUMMARY ---")for n, metrics in few_shot_results.items(): print(f"N = {n}:") print(f" Test F1 (Depression): {metrics['test_f1_depression']:.4f}") print(f" Test F1 (Weighted): {metrics['test_f1_weighted']:.4f}") print(f" Test Accuracy: {metrics['test_accuracy']:.4f}") print("-" * 30)


--- 🚀 STARTING FEW-SHOT RUN: N=50 ---
Created dataset with 50 samples.
label
0    26
1    24
Name: count, dtype: int64
Calculated class weights for N=50: tensor([0.9615, 1.0417], device='cuda:0')


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Loaded English checkpoint: /kaggle/input/checkpoints/checkpoint-100000
Starting training for N=50...


Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted,F1 Control,F1 Depression
1,No log,0.781808,0.443611,0.397088,0.568395,0.217327
2,No log,0.879892,0.510462,0.393243,0.141274,0.657648
3,1.071400,0.721282,0.537044,0.448911,0.241156,0.666919


Training for N=50 complete. Evaluating on test set...


...

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Loaded English checkpoint: /kaggle/input/checkpoints/checkpoint-100000
Starting training for N=1000...


Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted,F1 Control,F1 Depression
1,0.306600,0.345166,0.865377,0.864555,0.855733,0.873813
2,0.244500,0.234438,0.912883,0.912800,0.910998,0.914691
3,0.260000,0.184796,0.931307,0.931313,0.932593,0.929970
4,0.170000,0.202711,0.933675,0.933659,0.935714,0.931503


Training for N=1000 complete. Evaluating on test set...



--- Classification Report for N=1000 ---
              precision    recall  f1-score   support

     control       0.93      0.95      0.94      4579
  depression       0.95      0.93      0.94      4362

    accuracy                           0.94      8941
   macro avg       0.94      0.94      0.94      8941
weighted avg       0.94      0.94      0.94      8941


--- Summary Results for N=1000 ---
{'test_loss': 0.19978263974189758, 'test_accuracy': 0.9390448495693994, 'test_f1_weighted': 0.9390240593706555, 'test_f1_control': 0.9410619660430409, 'test_f1_depression': 0.9368847712796757, 'test_runtime': 173.0188, 'test_samples_per_second': 51.676, 'test_steps_per_second': 3.231}
--- ✅ FINISHED RUN: N=1000 ---


--- 🏆 FINAL FEW-SHOT RESULTS SUMMARY ---
N = 50:
  Test F1 (Depression): 0.6650
  Test F1 (Weighted):   0.4395
  Test Accuracy:        0.5322
------------------------------
N = 100:
  Test F1 (Depression): 0.7839
  Test F1 (Weighted):   0.7691
  Test Accuracy:        0.7704
-

## Full-Shot Prediction

In [18]:
FULL_SHOT_OUTPUT_DIR = "/kaggle/working/xlm-roberta-full-shot/"

In [19]:
# Cell 18.5 (Insert before full-shot training)print("🔄 Reloading fresh model with English adapter for full-shot training...")# 1. Load base modelmodel = XLMRobertaForSequenceClassification.from_pretrained( "xlm-roberta-base", num_labels=train_df['label'].nunique())# 2. Define LoRA configpeft_config = LoraConfig( task_type=TaskType.SEQ_CLS, r=8, lora_alpha=16, lora_dropout=0.1, target_modules=['query', 'value', 'key', 'dense'])model = get_peft_model(model, peft_config)# 3. Load English adaptermodel.load_adapter(ENGLISH_CHECKPOINT_PATH, adapter_name="default")model.to(device)print(" Fresh English checkpoint loaded for full-shot training")# 4. CALCULATE CLASS WEIGHTS (for the *full* train set)print(" Calculating class weights for the full training set...")train_labels = train_df['label'].tolist()class_weights = compute_class_weight( class_weight='balanced', classes=np.unique(train_labels), y=train_labels)class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)print(f"Calculated class weights: {class_weights_tensor}")# 6. SET FULL-SHOT TRAINING ARGUMENTStraining_args = TrainingArguments( output_dir=FULL_SHOT_OUTPUT_DIR, num_train_epochs=3, # Standard 3-epoch run per_device_train_batch_size=16, gradient_accumulation_steps=2, logging_steps=500, eval_strategy="steps", eval_steps=1000, save_strategy="steps", save_steps=1000, load_best_model_at_end=True, metric_for_best_model="eval_f1_depression", greater_is_better=True, fp16=True, report_to="none" # Set to "wandb" if you want to log)# 7. INITIALIZE AND TRAINtrainer = WeightedLossTrainer( model=model, args=training_args, train_dataset=train_dataset, # <-- Using the FULL train_dataset eval_dataset=val_dataset, compute_metrics=compute_metrics, class_weights_tensor=class_weights_tensor)print(" Starting FULL-SHOT training run...")# This starts a *new* run from step 0, using the English weightstrainer.train()print(" Full-shot training complete!")# 8. EVALUATE AND SAVEprint(" Evaluating on the test set...")prediction_output = trainer.predict(test_dataset)# Get the reportreport_str, _, _ = get_classification_report( prediction_output, target_names=['control', 'depression'])print("\n--- Full-Shot Performance Report ---")print(report_str)# Save the reportreport_path = os.path.join(FULL_SHOT_OUTPUT_DIR, "final_classification_report.txt")with open(report_path, "w") as f: f.write("--- Full-Shot Performance Report ---\n") f.write(report_str)# Save the final modeltrainer.save_model(os.path.join(FULL_SHOT_OUTPUT_DIR, "best_model"))print(f" Best model and report saved to {FULL_SHOT_OUTPUT_DIR}")

🔄 Reloading fresh model with English adapter for full-shot training...


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


✅ Fresh English checkpoint loaded for full-shot training
✅ Calculating class weights for the full training set...
Calculated class weights: tensor([0.9764, 1.0248])
🚀 Starting FULL-SHOT training run...


Step,Training Loss,Validation Loss,Accuracy,F1 Weighted,F1 Control,F1 Depression
1000,0.118600,0.101525,0.964732,0.964731,0.964690,0.964774
2000,0.087500,0.085744,0.970654,0.970657,0.970853,0.970452


🎉 Full-shot training complete!
🚀 Evaluating on the test set...



--- Full-Shot Performance Report ---
              precision    recall  f1-score   support

     control       0.99      0.95      0.97      4579
  depression       0.95      0.99      0.97      4362

    accuracy                           0.97      8941
   macro avg       0.97      0.97      0.97      8941
weighted avg       0.97      0.97      0.97      8941

✅ Best model and report saved to /kaggle/working/xlm-roberta-full-shot/
